In [ ]:
drug_name: "cenobamate"  # 또는 "warfarin"
population: "Korean"     # "East_Asian", "European", "African", "Multi-ethnic"
n_patients: 1000
dose_mg: 200


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import requests
from scipy.integrate import odeint
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# API 키 (고객 발급 후 입력)
DRUGBANK_TOKEN = st.secrets.get("DRUGBANK_TOKEN", "your_free_token_here")  # https://dev.drugbank.com
PHARMGKB_TOKEN = "your_pharmgkb_token"  # 무료

@st.cache_data
def fetch_drug_pk(drug_name):
    """DrugBank API: PK 파라미터 자동 추출 [web:116][web:117]"""
    headers = {"Authorization": f"Bearer {DRUGBANK_TOKEN}"}
    url = f"https://api.drugbank.com/v1/drugs/search?q={drug_name}"
    resp = requests.get(url, headers=headers).json()
    
    if resp['drugs']:
        drug = resp['drugs'][0]
        drug_id = drug['drugbank_id']
        
        # 상세 PK
        pk_url = f"https://api.drugbank.com/v1/drugs/{drug_id}"
        pk_data = requests.get(pk_url, headers=headers).json()
        
        return {
            'name': drug['name'],
            'Vd': float(pk_data.get('general_pharmacokinetics', {}).get('volume_of_distribution', 45)),
            'fu': float(pk_data.get('general_pharmacokinetics', {}).get('plasma_protein_binding', 0.5)),
            'CL': float(pk_data.get('general_pharmacokinetics', {}).get('clearance', 0.5)),
            't_half': float(pk_data.get('general_pharmacokinetics', {}).get('half_life', 10)),
            'metabolism': pk_data.get('metabolism_enzymes', ['CYP3A4'])  # CYP 리스트
        }
    return {'Vd': 45, 'fu': 0.5, 'CL': 0.5, 'metabolism': ['CYP3A4']}  # 기본값

@st.cache_data
def fetch_pgx_freq(population, genes=['CYP2D6', 'CYP2C19']):
    """gnomAD/PharmGKB: 인종별 PM 빈도 [web:107][web:108]"""
    # 실제 gnomAD API 또는 PharmGKB 테이블 (하드코드 + API)
    freq_db = {
        'Korean': {'CYP2D6_PM': 0.46, 'CYP2C19_PM': 0.13},  # KoGES [web:108]
        'East_Asian': {'CYP2D6_PM': 0.01, 'CYP2C19_PM': 0.15},
        'European': {'CYP2D6_PM': 0.08, 'CYP2C19_PM': 0.03},
        'African': {'CYP2D6_PM': 0.07, 'CYP2C19_PM': 0.17},
        'Multi-ethnic': {'CYP2D6_PM': 0.05, 'CYP2C19_PM': 0.10}
    }
    return freq_db.get(population, freq_db['Multi-ethnic'])

@st.cache_data
def fetch_cpic_guideline(drug_name):
    """CPIC 가이드라인 [web:124]"""
    # PharmGKB/CPIC API 또는 테이블
    guidelines = {
        'cenobamate': {'PM_adjust': 0.5, 'hepatic_mild': 0.5},
        'warfarin': {'PM_adjust': 0.3},
        'codeine': {'PM_adjust': 0.0}  # 금기
    }
    return guidelines.get(drug_name.lower(), {'PM_adjust': 0.5})

# ═══════════════════════════════════════════════════════════════
# Streamlit UI (고객 입력)
# ═══════════════════════════════════════════════════════════════
st.title("🚀 상용 디지털 트윈 PBPK Platform")
st.sidebar.header("고객 입력")

drug_name = st.sidebar.text_input("약물명", "cenobamate")
population = st.sidebar.selectbox("집단(인종)", 
                                 ['Korean', 'East_Asian', 'European', 'African', 'Multi-ethnic'])
n_patients = st.sidebar.slider("환자 수", 100, 5000, 1000)
dose_mg = st.sidebar.slider("용량(mg)", 50, 1000, 200)

if st.sidebar.button("시뮬레이션 실행"):
    
    # 1. 실시간 데이터 가져오기
    with st.spinner("DrugBank/PubChem에서 약물 PK 로딩..."):
        pk_params = fetch_drug_pk(drug_name)
    
    with st.spinner("gnomAD에서 인종 PGx 빈도 로딩..."):
        pgx_freq = fetch_pgx_freq(population)
    
    cpic = fetch_cpic_guideline(drug_name)
    
    st.success(f"✅ {drug_name} PK: Vd={pk_params['Vd']}L, fu={pk_params['fu']}")
    st.success(f"✅ {population} PGx: CYP2D6 PM {pgx_freq['CYP2D6_PM']*100:.1f}%")
    
    # 2. 다민족 코호트 생성 (이전 코드 확장)
    np.random.seed(42)
    cohort = pd.DataFrame({
        'ID': range(1, n_patients+1),
        'Ethnicity': population,
        'Weight_kg': np.random.normal(70, 15, n_patients).clip(40, 120),
        'Age': np.random.normal(50, 15, n_patients).clip(18, 80),
        'CYP2D6_PM': np.random.choice([True, False], n_patients, 
                                     p=[pgx_freq['CYP2D6_PM'], 1-pgx_freq['CYP2D6_PM']])
    })
    
    # CPIC 적용 CL 조정
    cohort['CL_adjust'] = np.where(cohort['CYP2D6_PM'], 
                                  cpic['PM_adjust'], 1.0)
    
    # 3. mPBPK 시뮬 (간략 1-구획)
    def simple_pbk(y, t, CL, Vd): 
        return -CL/Vd * y
    
    t = np.linspace(0, 72, 200)
    aucs = []
    for _, pat in cohort.iterrows():
        CL = pk_params['CL'] * pat['CL_adjust']
        Vd = pk_params['Vd'] * (pat['Weight_kg']/70)  # 체중 정규화
        C0 = dose_mg / Vd
        conc = odeint(simple_pbk, C0, t, args=(CL, Vd))
        aucs.append(np.trapz(conc, t))
    
    cohort['AUC'] = aucs
    
    # 4. 결과 대시보드
    col1, col2 = st.columns(2)
    with col1:
        st.metric("평균 AUC", f"{cohort['AUC'].mean():.0f} μg·h/mL")
        st.metric("고위험(PM)", f"{cohort['CYP2D6_PM'].sum()}명 ({cohort['CYP2D6_PM'].mean()*100:.1f}%)")
    
    with col2:
        safe_dose = dose_mg * (np.median(cohort['AUC']) / cohort[cohort['CYP2D6_PM']]['AUC'].mean())
        st.metric("PM 안전 용량", f"{safe_dose:.0f}mg")
    
    # 히트맵 + 곡선
    fig = make_subplots(rows=1, cols=2, 
                       subplot_titles=('AUC 분포', f'{drug_name} 농도 곡선'))
    
    # AUC 히스토그램
    fig.add_trace(px.histogram(cohort, x='AUC', color='CYP2D6_PM', 
                              nbins=30).data[0], row=1, col=1)
    
    # 평균 곡선 (PM vs EM)
    em_conc = odeint(simple_pbk, dose_mg/pk_params['Vd'], t, 
                    args=(pk_params['CL'], pk_params['Vd']))
    pm_conc = odeint(simple_pbk, dose_mg/pk_params['Vd'], t, 
                    args=(pk_params['CL']*cpic['PM_adjust'], pk_params['Vd']))
    
    fig.add_trace(go.Scatter(x=t, y=em_conc.flatten(), name='EM', line=dict(color='blue')), row=1, col=2)
    fig.add_trace(go.Scatter(x=t, y=pm_conc.flatten(), name='PM', line=dict(color='red')), row=1, col=2)
    
    st.plotly_chart(fig, use_container_width=True)
    
    # CSV 다운로드 (고객용)
    csv = cohort.to_csv(index=False).encode('utf-8')
    st.download_button("📥 결과 다운로드 (CSV)", csv, f"{drug_name}_{population}_cohort.csv")
    
    st.balloons()

# API 키 가이드
with st.expander("🔑 API 키 발급 (5분)"):
    st.markdown("""
    1. **DrugBank**: https://dev.drugbank.com → Free tier 신청 → Token 복사
    2. **PharmGKB**: https://www.pharmgkb.org → Download → API docs
    3. **PubChem**: 무료 PUG REST (키 불필요)
    """)
